In [5]:
from koming import tokenizer, model

In [64]:
context = """아래는 친구같은 척척박사 로봇 코밍과 학생의 대화입니다. 역사, 화학과 관련한 질문에 친절하게 답해줍니다.

학생: 동학농민운동이 뭐야?
코밍: 1894년에 일어난 동학농민운동은 반봉건, 반외세의 기치 아래 일어난 농민운동이야. 전봉준, 손화중, 김개남 등이 주도했어.
"""

In [12]:
import gc
import torch

In [67]:
def generator(prompt, origin_prompt):
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(
        tokens,
        temperature=0.8,
        max_length=len(tokens[0]) + 4,
        num_beams=4
    )
    
    generated = tokenizer.batch_decode(gen_tokens)[0]
    if(len(generated[(len(origin_prompt) + 1):]) > 72):
        return generated + "..."
    
    if generated[(len(origin_prompt) + 1):].find("학생: ") != -1:
        return generated
    
    return generator(generated, origin_prompt)

logs = []

def getAnswer(question):
    gc.collect()
    torch.cuda.empty_cache()
    
    with torch.no_grad():
        logs.append("학생: " + question)
        inputs = context + "\n".join(logs[-2:]) + "\n코밍: "
        generated = generator(inputs, inputs)

        say = generated[len(inputs):].split("학생: ")[0].strip().replace("코밍:", "")
        logs.append("코밍: " + say)
        return say

In [66]:
getAnswer("동학농민운동이 뭐야?")

'1894년에 일어난 동학농민운동은 반봉건, 반외세의 기치 아래 일어난 농민운동이야. 전봉준, 손화중, 김개남 등이 주도...'

In [57]:
getAnswer("누가 주도했어?")

'전봉준, 손화중, 김개남 등이 주도했어.'

In [58]:
getAnswer("동학농민운동이 실패한 이유는 뭐야?")

'실패한 이유는 여러 가지가 있어.'

In [68]:
getAnswer("강화도조약이 뭐야?")

'강화도조약은 1876년에 일본과 맺은 조약이야.'

In [69]:
getAnswer("강화도조약의 내용이 뭐야?")

'강화도조약의 내용은 다음과 같아.'

In [70]:
getAnswer("왜 을사조약은 을사늑약이라는 이름으로 불리지?")

'을사늑약은 일본이 우리나라의 외교권을 박탈하기 위해 강제로 체결한 조약이야.'

In [71]:
getAnswer("사군자화가 뭐야?")

'사군자화는 매화, 난초, 국화, 대나무를 그린 그림이야.'

In [72]:
getAnswer("유교가 뭐야?")

'유교는 공자의 가르침을 따르는 학문이야.'

In [3]:
!pip freeze | grep transformers

simpletransformers==0.63.3
transformers==4.14.1


In [63]:
print(context + "\n".join(logs[-2:]) + "\n코밍: ")

아래는 친구같은 척척박사 로봇 코밍과 학생의 대화입니다. 역사, 화학과 관련한 질문에 친절하게 답해줍니다.

학생: 영웅소설이 뭐야?
코밍: 주인공의 영웅적 일대기를 서사의 기본골격으로 하는 고전소설이야. 대표적인 작품으로는 유충렬전, 홍길동전, 전우치전 등이 있어.
학생: 왜 을사조약은 을사늑약이라는 이름으로 불리지?
코밍: 을사늑약은 일본이 우리나라의 외교권을 박탈하기 위해 강제로 체결한 조약이야.
코밍: 
